In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
from google.colab import files

In [ ]:
upload = files.upload()

In [ ]:
df = pd.read_csv(r"basketball_stat.csv")
df

## 포지션 지식

https://ssolviol.tistory.com/86

In [ ]:
corr_tmp = df.copy()
corr_tmp

In [ ]:
# corr_tmp = corr_tmp.drop('Player', axis=1)
corr_tmp["Pos"] = corr_tmp["Pos"].map({"C":0, "SG":1})
corr_tmp

In [ ]:
corr_tmp.corr()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
sns.pairplot(df, 
             diag_kind='kde',
             hue="Pos", 
             palette='bright') 
plt.savefig(r"pairplot.png")

In [ ]:
tmp = df[["Pos","3P","BLK"]]
tmp

In [ ]:
tmp.columns

In [ ]:
tmp.Pos

## 선형회귀와 정규화

http://hleecaster.com/ml-linear-regression-concept/  
http://hleecaster.com/ml-normalization-concept/

## Logistic regression 사용

https://nittaku.tistory.com/478  
https://todayisbetterthanyesterday.tistory.com/11  
http://hleecaster.com/ml-logistic-regression-example/  
http://hleecaster.com/ml-logistic-regression-concept/  

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
tmp["Pos"] = tmp["Pos"].map({"C":0, "SG":1})

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tmp[["3P","BLK"]], tmp.Pos, test_size=0.2, random_state=10)

In [ ]:
sc = StandardScaler()
sc.fit(X_train)

X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [ ]:
y_train

In [ ]:
model = LogisticRegression(solver="lbfgs", C = 100)
model.fit(X_train_std, y_train)

In [ ]:
print("학습 데이터 점수: {}".format(model.score(X_train_std, y_train)))
print("평가 데이터 점수: {}".format(model.score(X_test_std, y_test)))

In [ ]:
X = np.vstack((X_train_std, X_test_std))
y = np.hstack((y_train, y_test))

In [ ]:
from matplotlib.colors import ListedColormap

In [ ]:
x1_min, x1_max = X[:,0].min()-1, X[:,0].max()+1
x2_min, x2_max = X[:,1].min()-1, X[:,1].max()+1
xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, 0.02),
                       np.arange(x2_min, x2_max, 0.02))

Z = model.predict(np.array([xx1.ravel(), xx2.ravel()]).T)

Z = Z.reshape(xx1.shape)

pos = ("C", "SG")
markers = (".", "^")
colors = ("blue", "red")
cmap = ListedColormap(colors[:len(np.unique(y))])

plt.figure(figsize = (20,10))
plt.contourf(xx1, xx2, Z, alpha=0.3, cmap = cmap)
plt.xlim(xx1.min(), xx1.max())
plt.ylim(xx2.min(), xx2.max())


for idx, cl in enumerate(np.unique(y)):
    plt.scatter(x=X[y==cl, 0], y=X[y==cl, 1],
                alpha=0.8, c=colors[idx],
                marker=markers[idx], label=pos[cl],
                edgecolor="b", s=300)



plt.tight_layout()
plt.savefig("logistic.png");

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
pred = model.predict(X_test)
print("Accuarcy : " + str(accuracy_score(y_test.values.ravel(), pred)))

## SVM 사용

https://todayisbetterthanyesterday.tistory.com/32  
https://bskyvision.com/163  
https://bskyvision.com/851  

In [ ]:
df_copy = df.copy()
df_copy

In [ ]:
import sklearn.svm as svm
import sklearn.metrics as mt
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
df_copy["Pos"] = df_copy["Pos"].map({"C":0, "SG":1})

In [ ]:
svm_X_train, svm_X_test, svm_y_train, svm_y_test = train_test_split(df_copy[['3P', 'BLK']], df_copy.Pos, test_size=0.2, random_state=11)

In [ ]:
svm_X_train

In [ ]:
standard = StandardScaler()

standard.fit(svm_X_train)

svm_X_train_std = standard.transform(svm_X_train)
svm_X_test_std = standard.transform(svm_X_test)

In [ ]:
from sklearn.svm import SVC

svm_model = SVC(kernel="rbf", C=10, gamma=0.2)

svm_model.fit(svm_X_train_std, svm_y_train)

In [ ]:
svm_y_pred = svm_model.predict(svm_X_test_std)
print("예측된 라벨: ",svm_y_pred)
print("Ground_Truth 라벨: ",svm_y_test)

In [ ]:
print("prediction accuracy : {:2f}".format(np.mean(svm_y_pred == svm_y_test)))

In [ ]:
df_copy

In [ ]:
def make_meshgrid(x, y, h=.02):
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy

def plot_contours(clf, xx, yy, **params):
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = plt.contourf(xx, yy, Z, **params)
    return out
 

In [ ]:
svm_X = df_copy[['3P', 'BLK']]
svm_y = df_copy.Pos

svm_X0, svm_X1 = svm_X['3P'], svm_X['BLK']
svm_xx, svm_yy = make_meshgrid(svm_X0, svm_X1)

plt.figure(figsize=(20,10))
plot_contours(svm_model,svm_xx, svm_yy, cmap=plt.cm.coolwarm, alpha=0.8)
plt.scatter(svm_X0,svm_X1,c=svm_y, cmap=plt.cm.coolwarm, s=200, edgecolors="k")
plt.xlim(svm_xx.min(), svm_xx.max())
plt.ylim(svm_yy.min(), svm_yy.max())
plt.xlabel("3P")
plt.ylabel("BLK")
plt.xticks([])
plt.yticks([])
plt.show()



## KNN 사용

https://onejunu.tistory.com/3

In [ ]:
!pip install mglearn

In [ ]:
import mglearn


In [ ]:
mglearn.plots.plot_knn_classification(n_neighbors=3)

In [ ]:
mglearn.plots.plot_knn_classification(n_neighbors=4)

In [ ]:
mglearn.plots.plot_knn_classification(n_neighbors=2)

In [ ]:
df.Pos.value_counts()

속성 설명

Player : 플레이어의 이름

Pos: 플레이어의 포지션을 나타냅니다. 'SG' 는 '슈팅가드' 'C'는 '센터'를 의미

3P : 평균 한 경기 3점슛 성공 횟수

2P : 평균 한 경기 2점슛 성공 횟수

TRB: 평균 한경기 리바운드 성공 횟수

AST: 평균 한경기 어시스트 성공 횟수

STL: 평균 한경기 스틸 성공 횟수

BLK: 평균 한경기 블로킹 성공 횟수

 

5번줄 = Pos라는 속성의 각 속성의 이름과 개수를 반환합니다.

 

In [ ]:
df_knn = df.copy()
df_knn['Pos'] = df_knn['Pos'].map({"C":0, "SG": 1})
df_knn = df_knn[['Pos', '3P', 'BLK']]
df_knn

In [ ]:
knn_X_train, knn_X_test, knn_y_train, knn_y_test = train_test_split(df_knn[['3P','BLK']],df_knn.Pos ,test_size=0.2, random_state=11)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
standard = StandardScaler()

standard.fit(knn_X_train)

knn_X_train_std = standard.transform(knn_X_train)
knn_X_test_std = standard.transform(knn_X_test)

In [ ]:
max_k_range = knn_X_train.shape[0] // 2
k_list=[]

for i in range(3, max_k_range, 2):
    k_list.append(i)

cross_validation_scores=[]

for k in k_list:
    knn = KNeighborsClassifier(n_neighbors = k)
    scores = cross_val_score(knn, knn_X_train, knn_y_train.values.ravel(), cv=10, scoring="accuracy")
    
    cross_validation_scores.append(scores.mean())

print(cross_validation_scores)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(k_list, cross_validation_scores)
plt.xlabel("The Number of k")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
k = k_list[cross_validation_scores.index(max(cross_validation_scores))]
print("The best numver of k is ",k)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(knn_X_train, knn_y_train.values.ravel())

knn_pred = knn.predict(knn_X_test)

In [ ]:
knn_pred

In [ ]:
from sklearn.metrics import accuracy_score
print("accuracy : " + str(accuracy_score(knn_y_test.values.ravel(), knn_pred)))

In [ ]:

plt.figure(figsize=(20,10))

mglearn.plots.plot_2d_classification(knn, knn_X_train_std, fill=True)
mglearn.discrete_scatter(knn_X_train_std[:,0],knn_X_train_std[:,1],  knn_y_train)

In [ ]:
scale=100
xmax = knn_X_train_std[:][0].max()+1
xmin = knn_X_train_std[:][0].min()-1
ymax = knn_X_test_std[:][1].max()+1
ymin = knn_X_test_std[:][1].min()-1

xx = np.linspace(xmin,xmax,scale)
yy = np.linspace(ymin,ymax,scale)
data1, data2 = np.meshgrid(xx,yy)
X_grid = np.c_[data1.ravel(), data2.ravel()]
pred_y = knn.predict(X_grid)

fig=plt.figure(figsize=[10,10])

plt.imshow(pred_y.reshape(scale,scale), interpolation=None, origin='lower',
                extent=[xmin,xmax,ymin,ymax], alpha=0.5, cmap='gray_r')

plt.scatter(knn_X_train_std[:,0], knn_X_train_std[:,1], c=knn_y_train)


 








.

.

.

.

## DicisionTree 사용

https://tensorflow.blog/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D/2-3-5-%EA%B2%B0%EC%A0%95-%ED%8A%B8%EB%A6%AC/

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt_df = df.copy()
dt_df['Pos'] = dt_df['Pos'].map({"C": 0, "SG": 1})
dt_df = dt_df[['Pos', '3P', 'BLK']]
dt_train_data = dt_df[['3P','BLK']]
dt_train_label = dt_df['Pos']

In [ ]:
dt_df

In [ ]:
dt_model = DecisionTreeClassifier()

dt_X_train, dt_X_test, dt_y_train, dt_y_test = train_test_split(dt_df[['3P','BLK']], dt_df['Pos'], random_state = 10, test_size=0.2)

In [ ]:
dt_model.fit(dt_X_train, dt_y_train)

In [ ]:
dt_pred = dt_model.predict(dt_X_test)
accuracy_score(dt_y_test, dt_pred)

### 결정트리 시각화

In [ ]:
from sklearn.tree import export_graphviz
import pydot
import cv2

In [ ]:
export_graphviz(dt_model, out_file="dt.dot", class_names=["Center", "Shooting Guard"], feature_names=["3P", "BLK"], impurity=False, filled=True)

In [ ]:
(graph, ) = pydot.graph_from_dot_file("dt.dot", encoding="utf-8")

In [ ]:
graph.write_png("dt.png")

In [ ]:
dt_tree_img = cv2.imread("dt.png")
dt_tree_img = cv2.cvtColor(dt_tree_img, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(20,10))
plt.xticks([])
plt.yticks([])
plt.imshow(dt_tree_img)

## Stratified K-Fold 교차검증

https://continuous-development.tistory.com/166

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
skf = StratifiedKFold(n_splits=5)

In [ ]:
dt_train_data

In [ ]:
dt_train_label

In [ ]:
result_skfold = StratifiedKFold(n_splits=5)
idx_iter=0
cv_accuracy=[]

for train_index, test_index  in result_skfold.split(dt_train_data, dt_train_label):
    skf_X_train = dt_train_data.iloc[train_index]
    skf_X_test =dt_train_data.iloc[test_index]
    skf_y_train = dt_train_label.iloc[train_index]
    skf_y_test =  dt_train_label.iloc[test_index]
   
    
    dt_model.fit(skf_X_train, skf_y_train)
    skf_pred = dt_model.predict(skf_X_test)
    
    idx_iter += 1
    skf_accuracy = np.round(accuracy_score(skf_y_test, skf_pred), 4)
    train_size = skf_X_train.shape[0]
    test_size = skf_X_test.shape[0]
    
    print("#{0} 교차 검증 정확도 : {1}\n학습 데이터 크기 : {2}\n검증 데이터 크기: {3}".format(idx_iter, skf_accuracy, train_size, test_size))
    print("#{0} 검증 세트 인덱스: {1}\n\n".format(idx_iter, test_index))
    cv_accuracy.append(skf_accuracy)

## Tensorflow 활용

In [ ]:
import pandas as pd
import cv2
import io
import os
from google.colab import files
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import SGD, Adam

In [ ]:
df.drop('Player',axis=1, inplace=True)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
df["Pos"] = df["Pos"].map({"C":0, "SG":1})

In [ ]:
X_train, Y_train = df[['3P','BLK','TRB']], df['Pos']

In [ ]:
sc = StandardScaler()
sc.fit(X_train)

X_train_std = sc.transform(X_train)

In [ ]:
model = Sequential()

model.add(Flatten(input_shape=(3,)))
model.add(Dense(3, activation="sigmoid"))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer=SGD(learning_rate=0.01),loss="binary_crossentropy", metrics=['accuracy'])
model.summary()

In [ ]:
hist = model.fit(X_train_std, Y_train, epochs=2500, validation_split=0.2, verbose=2)

In [ ]:
model.evaluate(X_train_std, Y_train)

In [ ]:
plt.title("Loss")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.grid()

plt.plot(hist.history['loss'], label="train loss")
plt.plot(hist.history['val_loss'], label="validation loss")

plt.legend(loc="best")

plt.show()

In [ ]:
plt.title("Accuracy")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.grid()

plt.plot(hist.history['accuracy'], label="train accuracy")
plt.plot(hist.history['val_accuracy'], label="validation accuracy")

plt.legend(loc="best")
plt.show()

# 네이버 크롤러

In [ ]:
import os
import sys
import urllib.request
import datetime
import time
import json

client_id = '9Mqf9Xlt5E9ZhNg1NSNU'
client_secret = 'nV3dY5nPfm'


#[CODE 1]
def getRequestUrl(url):    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

#[CODE 2]
def getNaverSearch(node, srcText, start, display):    
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)
    
    url = base + node + parameters    
    responseDecode = getRequestUrl(url)   #[CODE 1]
    
    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

#[CODE 3]
def getPostData(post, jsonResult, cnt):    
    title = post['title']
    image = post['image']
    author = post['author']
    link = post['link']
    price = post['price']
    discount = post['discount']
    description = post['description']
    pubdate = post['pubdate']
    
    jsonResult.append({'cnt':cnt, 'image':image, 'title':title, 'author': author,
                       'link': link, 'price':price, 'discount':discount,
                       'description':description, 'pubdate':pubdate })
    return    

#[CODE 0]
def main():
    node = 'book'   # 크롤링 할 대상 =  책
    srcText = input('검색어를 입력하세요: ')
    cnt = 0
    jsonResult = []

    jsonResponse = getNaverSearch(node, srcText, 1, 100)  #[CODE 2]
    total = jsonResponse['total']
 
    while ((jsonResponse != None) and (jsonResponse['display'] != 0)):         
        for post in jsonResponse['items']:
            cnt += 1
            getPostData(post, jsonResult, cnt)  #[CODE 3]       
        
        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = getNaverSearch(node, srcText, start, 100)  #[CODE 2]
       
    print('전체 검색 : %d 건' %total)
    
    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult,  indent=4, sort_keys=True,  ensure_ascii=False)
                        
        outfile.write(jsonFile)
        
    print("가져온 데이터 : %d 건" %(cnt))
    print ('%s_naver_%s.json SAVED' % (srcText, node))
    
if __name__ == '__main__':
    main()

# 공공 데이터 API

In [ ]:
import os
import sys
import urllib.request
import datetime
import time
import json
import pandas as pd

ServiceKey="Ody77GLuYeR%2FeFqbpduMN2Bi4Cka2fztbgnj6E2Eux1kUhy3e4epR28XKBUaObiqPoVzAizxXMBPXtMyuC9v9Q%3D%3D"

#[CODE 1]
def getRequestUrl(url):    
    req = urllib.request.Request(url)    
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None


#[CODE 2]
def getTourismStatsItem(yyyymm, national_code, ed_cd):    
    service_url = "http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList"
    parameters = "?_type=json&serviceKey=" + ServiceKey   #인증키
    parameters += "&YM=" + yyyymm
    parameters += "&NAT_CD=" + national_code
    parameters += "&ED_CD=" + ed_cd
    url = service_url + parameters
    
    retData = getRequestUrl(url)   #[CODE 1]
    
    if (retData == None):
        return None
    else:
         return json.loads(retData)

#[CODE 3]
def getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear):
    jsonResult = []
    result = []
    natName=''
    dataEND = "{0}{1:0>2}".format(str(nEndYear), str(12)) #데이터 끝 초기화
    isDataEnd = 0 #데이터 끝 확인용 flag 초기화    
    
    for year in range(nStartYear, nEndYear+1):        
        for month in range(1, 13):
            if(isDataEnd == 1): break #데이터 끝 flag 설정되어있으면 작업 중지.
            yyyymm = "{0}{1:0>2}".format(str(year), str(month))            
            jsonData = getTourismStatsItem(yyyymm, nat_cd, ed_cd) #[CODE 2]
            
            if (jsonData['response']['header']['resultMsg'] == 'OK'):               
                # 입력된 범위까지 수집하지 않았지만, 더이상 제공되는 데이터가 없는 마지막 항목인 경우 -------------------
                if jsonData['response']['body']['items'] == '': 
                    isDataEnd = 1 #데이터 끝 flag 설정
                    dataEND = "{0}{1:0>2}".format(str(year), str(month-1))
                    print("데이터 없음.... \n 제공되는 통계 데이터는 %s년 %s월까지입니다."                          
                          %(str(year), str(month-1)))                    
                    break                
                #jsonData를 출력하여 확인......................................................
                print (json.dumps(jsonData, indent=4, 
                         sort_keys=True, ensure_ascii=False))          
                natName = jsonData['response']['body']['items']['item']['natKorNm']
                natName = natName.replace(' ', '')
                num = jsonData['response']['body']['items']['item']['num']
                ed = jsonData['response']['body']['items']['item']['ed']
                print('[ %s_%s : %s ]' %(natName, yyyymm, num))
                print('----------------------------------------------------------------------')                
                jsonResult.append({'nat_name': natName, 'nat_cd': nat_cd,
                                 'yyyymm': yyyymm, 'visit_cnt': num})
                result.append([natName, nat_cd, yyyymm, num])
                
    return (jsonResult, result, natName, ed, dataEND)

#[CODE 0]
def main():
    jsonResult = []
    result = []
    natName=''
    print("<< 국내 입국한 외국인의 통계 데이터를 수집합니다. >>")
    nat_cd = input('국가 코드를 입력하세요(중국: 112 / 일본: 130 / 미국: 275) : ')
    nStartYear =int(input('데이터를 몇 년부터 수집할까요? : '))
    nEndYear = int(input('데이터를 몇 년까지 수집할까요? : '))
    ed_cd = "E"     #E : 방한외래관광객, D : 해외 출국
    
    jsonResult, result, natName, ed, dataEND =getTourismStatsService(nat_cd,
                                            ed_cd, nStartYear, nEndYear) #[CODE 3]

    if (natName=='') : #URL 요청은 성공하였지만, 데이터 제공이 안된 경우
        print('데이터가 전달되지 않았습니다. 공공데이터포털의 서비스 상태를 확인하기 바랍니다.')
    else:
        #파일저장 1 : json 파일       
        with open('./%s_%s_%d_%s.json' % (natName, ed, nStartYear, dataEND), 'w', 
                    encoding='utf8') as outfile:
            jsonFile  = json.dumps(jsonResult, indent=4, sort_keys=True, ensure_ascii=False)
            outfile.write(jsonFile)
        #파일저장 2 : csv 파일   
        columns = ["입국자국가", "국가코드", "입국연월", "입국자 수"]
        result_df = pd.DataFrame(result, columns = columns)
        result_df.to_csv('./%s_%s_%d_%s.csv' % (natName, ed, nStartYear, dataEND),
                    index=False, encoding='cp949')
    
if __name__ == '__main__':
    main()

# BS4와 셀레니움

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import datetime

In [ ]:
naver_url = 'https://www.naver.com'

In [ ]:
html = urllib.request.urlopen(naver_url)
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# print(soup.prettify())

할리스

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import datetime

#[CODE 1]
def hollys_store(result):
    for page in range(1,59):
        Hollys_url = 'https://www.hollys.co.kr/store/korea/korStore.do?pageNo=%d&sido=&gugun=&store=' %page
        print(Hollys_url)
        html = urllib.request.urlopen(Hollys_url)
        soupHollys = BeautifulSoup(html, 'html.parser')
        tag_tbody = soupHollys.find('tbody')
        for store in tag_tbody.find_all('tr'):
            if len(store) <= 3:
                break
            store_td = store.find_all('td')
            store_name = store_td[1].string
            store_sido = store_td[0].string
            store_address = store_td[3].string
            store_phone = store_td[5].string
            result.append([store_name]+[store_sido]+[store_address]
                          +[store_phone])
    return

#[CODE 0]
def main():
    result = []
    print('Hollys store crawling >>>>>>>>>>>>>>>>>>>>>>>>>>')
    hollys_store(result)   #[CODE 1] 호출 
    hollys_tbl = pd.DataFrame(result, columns=('store', 'sido-gu', 'address','phone'))
    hollys_tbl.to_csv('hollys.csv', encoding='utf-8', mode='w', index=True)
    del result[:]
       
if __name__ == '__main__':
     main()

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import datetime
import requests

#[CODE 1]
def hollys_store(result):
    for page in range(1,50):
        Hollys_url = "https://gall.dcinside.com/board/lists/?id=football_new7&page=%d&exception_mode=recommend"%page
        headers={"User-Agent" : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36'}
        print(Hollys_url)
        html = requests.get(Hollys_url,headers = headers)
        soupHollys = BeautifulSoup(html.text, 'html.parser')
        tag_tbody = soupHollys.find('tbody')
        # print(tag_tbody)
        for store in tag_tbody.find_all('tr'):
            if len(store) <= 3:
                break
            store_td = store.find_all('td')
            store_name = store_td[1].get_text()
            store_sido = store_td[0].string
            store_address = store_td[2].get_text()
            store_phone = store_td[3].string

            result.append([store_name]+[store_sido]+[store_address]
                          +[store_phone])
    return

#[CODE 0]
def main():
    result = []
    print('Hollys store crawling >>>>>>>>>>>>>>>>>>>>>>>>>>')
    hollys_store(result)   #[CODE 1] 호출 
    hollys_tbl = pd.DataFrame(result, columns=('store', 'sido-gu', 'address','phone'))
    hollys_tbl.to_csv('hae.csv', encoding='utf-8', mode='w', index=True)
    del result[:]
       
if __name__ == '__main__':
     main()

In [ ]:
|#block-views-block-8dae-bunlyu-block-3 > div > div > div > div

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import datetime

#[CODE 1]
def pascucci_store(result):
    for page in range(1,53):
        pascucci_url = 'https://www.caffe-pascucci.co.kr/store/storeList.asp?page=%d' %page
        print(pascucci_url)
        html = urllib.request.urlopen(pascucci_url)
        soupHollys = BeautifulSoup(html, 'html.parser')
        tag_tbody = soupHollys.find('tbody')
        for store in tag_tbody.find_all('tr'):
            if len(store) <= 3:
                break
            store_td = store.find_all('td')
            # print(store_td)
            store_name = store_td[1].string
            store_sido = store_td[2].string
            store_address = store_td[4].select_one('p.addr').get_text()
            store_phone = store_td[5].select_one('span.tel').get_text()
            result.append([store_name]+[store_sido]+[store_address]
                          +[store_phone])
    return

#[CODE 0]
def main():
    result = []
    print('Hollys store crawling >>>>>>>>>>>>>>>>>>>>>>>>>>')
    pascucci_store(result)   #[CODE 1] 호출 
    hollys_tbl = pd.DataFrame(result, columns=('store', 'sido-gu', 'address','phone'))
    hollys_tbl.to_csv('pascu.csv', encoding='utf-8', mode='w', index=True)
    del result[:]
       
if __name__ == '__main__':
     main()

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import datetime

#[CODE 1]
def ai_hub_list(result):
    for page in ["vision", "natural-language", "education", "land", "primary-inderstry", "safty", "autonomous", "healthcare"]:
        aihub_url = 'https://aihub.or.kr/aihub-data/%s/about'%page
        print(aihub_url)
        html = urllib.request.urlopen(aihub_url)
        souphub = BeautifulSoup(html, 'html.parser')
        tag_ul = souphub.find_all("li", attrs={"class":"view-list-item"})
        # print(tag_ul)
        for post in tag_ul:
            category = post.find("span", attrs={"class" : "aidata_category_8"})
            print(category)

    return

#[CODE 0]
def main():
    result = []
    print('Hollys store crawling >>>>>>>>>>>>>>>>>>>>>>>>>>')
    ai_hub_list(result)   #[CODE 1] 호출 
    # hollys_tbl = pd.DataFrame(result, columns=('store', 'sido-gu', 'address','phone'))
    # hollys_tbl.to_csv('pascu.csv', encoding='utf-8', mode='w', index=True)
    # del result[:]
       
if __name__ == '__main__':
     main()